`PAIR PROGRAMMING LIMPIEZA I`

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.
¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
¿Cuál es el rango de edad que sufre la mayoría de los ataques?
Independientemente de la edad, sufren los hombres más ataques que las mujeres?
¿En qué mes ocurren más ataques?
¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
¿Cómo han evolucionado los ataques a lo largo del tiempo?
De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no:


In [1]:
#Importing the libraries that we will be using in this notebook.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime

In [2]:
#Reading the csv file and showing the first two rows.
df=pd.read_csv('../Datos/attacks_pandas7_copia.csv')
df.head(2)

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Age,Time,...,Siglo,Unnamed: 0_x,Sex,Species,href,Unnamed: 0_y,Injury,Mes,fatal_bien,genero_bien
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,...,siglo - XXI,NaN,NaN,NaN,NaN,0,"No injury to occupant, outrigger canoe and pad...",Jun,N,F
1,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,18,Late afternoon,...,siglo - XXI,NaN,NaN,NaN,NaN,6,FATAL,Jun,Y,M




Entre las tareas que tendréis que hacer hoy:

1. Al empezar a trabajar con este dataframe ya eliminamos algunas columnas que a priori no nos interesaban. Ahora llega el momento de eliminar alguna columna más. En este caso vamos tendréis que eliminar las columnas que no nos sean útiles para contestar a nuestras preguntas. Pero ojo ⚠️, haced una copia del dataframe para no "cargarnos" el dataframe original y perder la info.


In [14]:
#Replacing the spaces in the column names with underscores and making the column names lowercase.
nuevas_columnas = {col: col.replace(" ", "_").lower() for col in df.columns}
nuevas_columnas


{'case_number': 'case_number',
 'year': 'year',
 'type': 'type',
 'country': 'country',
 'area': 'area',
 'location': 'location',
 'activity': 'activity',
 'name': 'name',
 'age': 'age',
 'time': 'time',
 'species': 'species',
 'href': 'href',
 'siglo': 'siglo',
 'unnamed:_0_x': 'unnamed:_0_x',
 'sex_': 'sex_',
 'species_': 'species_',
 'unnamed:_0_y': 'unnamed:_0_y',
 'injury': 'injury',
 'mes': 'mes',
 'fatal_bien': 'fatal_bien',
 'genero_bien': 'genero_bien'}

In [15]:
df.rename(columns = nuevas_columnas, inplace = True)
df.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,...,siglo,unnamed:_0_x,sex_,species_,href,unnamed:_0_y,injury,mes,fatal_bien,genero_bien
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,...,siglo - XXI,NaN,NaN,NaN,NaN,0,"No injury to occupant, outrigger canoe and pad...",Jun,N,F
1,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,18,Late afternoon,...,siglo - XXI,NaN,NaN,NaN,NaN,6,FATAL,Jun,Y,M


In [5]:
#Creating a copy of the dataframe df.
df_copia = df.copy()

In [6]:
#Showing the columns of the dataframe.
df_copia.columns

Index(['case_number', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'age', 'time', 'species', 'href', 'siglo',
       'unnamed:_0_x', 'sex_', 'species_', 'href', 'unnamed:_0_y', 'injury',
       'mes', 'fatal_bien', 'genero_bien'],
      dtype='object')

In [7]:
#Creating a list of columns to be deleted.
columnas_eliminar=['sex_','unnamed:_0_x','unnamed:_0_y','href']

In [8]:
#Delete the columns
df_copia.drop(columnas_eliminar, axis=1, inplace=True)
df_copia.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,species,siglo,species_,injury,mes,fatal_bien,genero_bien
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,siglo - XXI,NaN,"No injury to occupant, outrigger canoe and pad...",Jun,N,F
1,2018.06.03.a,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,18,Late afternoon,Tiger shark,siglo - XXI,NaN,FATAL,Jun,Y,M


2. ¿Hay valores duplicados en nuestro dataframe? En caso de que los haya, eliminándolos.


In [9]:
df_copia.duplicated().sum()

0

3. Como hemos visto, algunas columnas no tienen el tipo de datos que deberían. Cambiad el tipo de dato para la columna de year.


In [11]:
#Showing the information of the dataframe.
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672 entries, 0 to 1671
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   case_number  1672 non-null   object 
 1   year         1672 non-null   float64
 2   type         1672 non-null   object 
 3   country      1662 non-null   object 
 4   area         1626 non-null   object 
 5   location     1621 non-null   object 
 6   activity     1641 non-null   object 
 7   name         1663 non-null   object 
 8   age          1518 non-null   object 
 9   time         1512 non-null   object 
 10  species      1502 non-null   object 
 11  siglo        1502 non-null   object 
 12  species_     44 non-null     object 
 13  injury       1664 non-null   object 
 14  mes          1518 non-null   object 
 15  fatal_bien   1573 non-null   object 
 16  genero_bien  1672 non-null   object 
dtypes: float64(1), object(16)
memory usage: 222.2+ KB


In [12]:
#Converting the column 'year' to an integer.
df_copia['year']=df['year'].astype('int64', errors='ignore')

In [13]:
#check
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672 entries, 0 to 1671
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   case_number  1672 non-null   object
 1   year         1672 non-null   int64 
 2   type         1672 non-null   object
 3   country      1662 non-null   object
 4   area         1626 non-null   object
 5   location     1621 non-null   object
 6   activity     1641 non-null   object
 7   name         1663 non-null   object
 8   age          1518 non-null   object
 9   time         1512 non-null   object
 10  species      1502 non-null   object
 11  siglo        1502 non-null   object
 12  species_     44 non-null     object
 13  injury       1664 non-null   object
 14  mes          1518 non-null   object
 15  fatal_bien   1573 non-null   object
 16  genero_bien  1672 non-null   object
dtypes: int64(1), object(16)
memory usage: 222.2+ KB


In [16]:

df['age']=df['age'].astype('int64', errors='ignore')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672 entries, 0 to 1671
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   case_number   1672 non-null   object 
 1   year          1672 non-null   float64
 2   type          1672 non-null   object 
 3   country       1662 non-null   object 
 4   area          1626 non-null   object 
 5   location      1621 non-null   object 
 6   activity      1641 non-null   object 
 7   name          1663 non-null   object 
 8   age           1518 non-null   object 
 9   time          1512 non-null   object 
 10  species       1502 non-null   object 
 11  href          1502 non-null   object 
 12  siglo         1502 non-null   object 
 13  unnamed:_0_x  170 non-null    float64
 14  sex_          156 non-null    object 
 15  species_      44 non-null     object 
 16  href          170 non-null    object 
 17  unnamed:_0_y  1672 non-null   int64  
 18  injury        1664 non-null 

4. En la columna de country poner todos los valores en minúsculas.
💡 Pista Tendréis que usar una función o una lambda 💡


In [17]:
#function that converts a string to lowercase.
def minusculas(x):
    try:
        if isinstance(x,str): #buscado en pep8
                return x.lower()
        else:
            return x
    except:
        return x

In [19]:
#Applying the function minusculas to the column country.
df['country']= df['country'].apply(minusculas)


5. Guardamos el csv para seguir trabajando en el siguiente ejercicio de pair de limpieza.